In [2]:
pip install geopandas

  Obtaining dependency information for geopandas from https://files.pythonhosted.org/packages/c4/64/7d344cfcef5efddf9cf32f59af7f855828e9d74b5f862eddf5bfd9f25323/geopandas-1.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for pyogrio>=0.7.2 from https://files.pythonhosted.org/packages/8d/2c/c761e6adeb81bd4029a137b3240e7214a8c9aaf225883356196afd6ef9d8/pyogrio-0.10.0-cp311-cp311-macosx_12_0_arm64.whl.metadata
  Obtaining dependency information for pyproj>=3.3.0 from https://files.pythonhosted.org/packages/2d/4d/610fe2a17de71b4fe210af69ce25f2d65379ba0a48299129894d0d0988ee/pyproj-3.7.0-cp311-cp311-macosx_14_0_arm64.whl.metadata
  Obtaining dependency information for shapely>=2.0.0 from https://files.pythonhosted.org/packages/37/63/e182e43081fffa0a2d970c480f2ef91647a6ab94098f61748c23c2a485f2/shapely-2.0.6-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 5.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import geopandas as gpd

# Sample data
data =  pd.read_excel('/Users/anjaliraj/Documents/Capstone/Final Data/Combined_Data.xlsx')
data.head()

,Estimated_market_value,Selling_Price,Date_Sold,Estimated_Monthly_Cost,Estimated_Rental_Value,Zipcode,Beds,Bathrooms,Square_Feet,Address,...,Estimated_Salesrange_max,Company_Adj,Stories,Roof,Laundry,Flooring,Cooling_Type,Garage_Spaces,Transit_Score,Bike_Score
0,2260062.0,1950000,2021-11-19,10536.0,5263.43,95070,3.0,2.0,1249.0,10612 Gascoigne Dr,...,2550000.0,Apple,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1723047.0,1850000,2022-05-17,8632.0,4115.00,95070,3.0,2.5,1598.0,20579 Cedarbrook Ter,...,1950000.0,Apple,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1629260.0,1685000,2022-07-02,8258.0,5374.13,95014,3.0,2.0,1487.0,20239 Northwest Sq,...,1840000.0,Apple,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1813091.0,1830000,2022-02-21,8968.0,4848.10,94024,3.0,2.5,1670.0,10280 Park Green Ln #850,...,2050000.0,Alphabet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1163353.0,1090000,2022-03-11,5342.0,3407.00,95015,3.0,2.0,1491.0,10090 Pasadena Ave Unit A4,...,1320000.0,Apple,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
from folium.plugins import HeatMap

# Select relevant columns
df = data[['Zipcode', 'Selling_Price']]

# Drop rows with missing values
df = df.dropna(subset=["Zipcode", "Selling_Price"])

# Ensure Zipcode is treated as a string (to match geoJSON format)
df['Zipcode'] = df['Zipcode'].astype(str)

# Group by Zipcode to calculate the average Selling_Price
zip_data = df.groupby("Zipcode")["Selling_Price"].mean().reset_index()

# Coordinates for each ZIP code (manually defined or using a lookup)
# For simplicity, let's use approximate coordinates for a few zip codes
zip_locations = {
    "94103": [37.7749, -122.4194],  # San Francisco
    "94588": [37.7017, -121.8893],  # Pleasanton
    "95054": [37.3849, -121.9692],  # Santa Clara
    "94105": [37.7890, -122.3912],  # San Francisco
    "95131": [37.3885, -121.8863],  # San Jose
    "95051": [37.3541, -121.9552],  # Santa Clara
    "95032": [37.2266, -121.9747],  # Los Gatos
    "94025": [37.4519, -122.1771],  # Menlo Park
    "94085": [37.3861, -122.0839],  # Sunnyvale
    "95134": [37.4143, -121.9524],  # San Jose
    "95014": [37.3230, -122.0322],  # Cupertino
    "94043": [37.3861, -122.0839],  # Mountain View
    "95110": [37.3382, -121.8863],  # San Jose
}

# Create a Folium map with a modern, lighter tile style (CartoDB positron)
m = folium.Map(location=[37.5, -122.0], zoom_start=9, tiles="CartoDB positron")

# Prepare the data for HeatMap
heat_data = []
for _, row in zip_data.iterrows():
    zipcode = row['Zipcode']
    if zipcode in zip_locations:
        lat, lon = zip_locations[zipcode]
        heat_data.append([lat, lon, row['Selling_Price']])

# Add the heatmap layer with an 'earthquake-like' appearance
HeatMap(heat_data, min_opacity=0.4, max_val=max(zip_data['Selling_Price']), radius=25, blur=15, 
        gradient={0: 'blue', 0.2: 'green', 0.4: 'yellow', 0.6: 'orange', 1: 'red'}).add_to(m)

# Save the map to an HTML file
m.save('/Users/anjaliraj/Documents/Capstone/selling_price_heatmap.html')

# Display the map
m


<ipython-input-3-b47a96aa7ee4>:45: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  HeatMap(heat_data, min_opacity=0.4, max_val=max(zip_data['Selling_Price']), radius=25, blur=15,
